In [2]:
import pandas as pd

In [28]:
df = pd.read_csv('Data\Mcdonald_review - 6 months - 40 stores.csv')

In [29]:
df.head()

,reviewer_id,store_address,latitude,longitude,rating_count,review_time,review,rating,sentiment
0,1,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star,negative
1,2,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars,positive
2,3,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star,negative
3,4,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was �����������...,5 stars,positive
4,5,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star,negative


In [36]:
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the dataframe by cleaning and aggregating data.
    
    Args:
        df (pd.DataFrame): The input dataframe.
    
    Returns:
        pd.DataFrame: The preprocessed dataframe.
    """
    # Create a copy of the dataframe to avoid SettingWithCopyWarning
    df = df.copy()
    
    # Define a mapping for star ratings
    rating_map = {
        '1 star': 1.0,
        '2 stars': 2.0,
        '3 stars': 3.0,
        '4 stars': 4.0,
        '5 stars': 5.0
    }
    
    # Map ratings using the defined dictionary
    df['rating'] = df['rating'].replace(rating_map)
    
    # Drop rows with NaN ratings
    df = df.dropna(subset=['rating'])
    
    # Convert latitude and longitude to numeric
    df['latitude'] = pd.to_numeric(df['latitude '], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    
    # Drop rows with NaN latitude or longitude
    df = df.dropna(subset=['latitude', 'longitude'])
    
    # Recalculate rating_count for each store_address
    df['rating_count'] = df.groupby('store_address')['rating'].transform('count')
    
    # Group by store_address and calculate weighted average rating
    df_grouped = df.groupby('store_address').agg({
        'latitude': 'first',
        'longitude': 'first',
        'rating_count': 'first',
        'rating': lambda x: round((x * df.loc[x.index, 'rating_count']).sum() / df.loc[x.index, 'rating_count'].sum(), 2) if df.loc[x.index, 'rating_count'].sum() != 0 else round(x.mean(), 2),
        'sentiment': lambda x: x.mode().iloc[0] if not x.empty else None
    }).reset_index()
    
    return df_grouped


In [37]:
df_pro = preprocess_dataframe(df)

C:\Users\York Yong\AppData\Local\Temp\ipykernel_32516\301355854.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['rating'] = df['rating'].replace(rating_map)


In [38]:
df_pro

,store_address,latitude,longitude,rating_count,rating,sentiment
0,"1044 US-11, Champlain, NY 12919, United States",44.981410,-73.459820,46,3.59,positive
1,"10451 Santa Monica Blvd, Los Angeles, CA 90025...",34.056593,-118.426000,2,5.00,positive
2,"10901 Riverside Dr, North Hollywood, CA 91602,...",34.152507,-118.367904,168,3.22,positive
3,"1100 N US Hwy 377, Roanoke, TX 76262, United S...",33.009318,-97.222925,69,3.29,positive
4,"111 Madison St, Oak Park, IL 60302, United States",41.879656,-87.777913,70,3.31,positive
5,"1121 Garnet Ave, San Diego, CA 92109, United S...",32.797661,-117.249470,42,2.31,negative
6,"11382 US-441, Orlando, FL 32837, United States",28.399986,-81.405103,46,3.02,positive
7,"114 Delancey St, New York, NY 10002, United St...",40.718838,-73.988280,3,3.33,neutral
8,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,68,3.00,positive
9,"1415 E State Rd, Fern Park, FL 32730, United S...",28.655350,-81.342692,163,2.87,negative


In [41]:
df_pro['rating'].rank(method='min', ascending=False)

0      5.0
1      1.0
2     17.0
3     15.0
4     14.0
5     36.0
6     22.0
7     12.0
8     23.0
9     27.0
10    39.0
11    15.0
12    35.0
13    31.0
14    23.0
15     3.0
16    30.0
17    11.0
18     6.0
19    10.0
20    33.0
21     4.0
22    29.0
23    17.0
24    12.0
25    32.0
26    19.0
27     8.0
28    26.0
29    34.0
30     9.0
31    20.0
32     7.0
33     2.0
34    21.0
35    25.0
36    38.0
37    28.0
38    37.0
Name: rating, dtype: float64